In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bcprojectii/NEAREST_BUS_STOP.csv


In [2]:
import datetime
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
data_path = '/kaggle/input/bcprojectii/NEAREST_BUS_STOP.csv'
data = pd.read_csv(data_path)
print(data.shape)
data.head()

(201956, 14)


,id,transaction_latitude,transaction_longitude,bus_stop_latitude,bus_stop_longitude,Stop Name,distance,Address,date,time.x,am_pm,bus_num,Route,movement
0,1,42.136107,-75.910827,42.136078,-75.910828,N. College/Pavilion,0.000029,288 N College Dr,8/21/23,1:18:38 PM,PM,101,8,Moving
1,2,42.101618,-75.910471,42.101609,-75.910658,BC Junction 8 Front St,0.000058,24 Prospect Ave (Landmarks: BC JUNCTION),8/21/23,1:51:43 PM,PM,101,8,Moving
2,3,42.162560,-75.892693,42.162544,-75.892517,Front / Quinn,0.000046,1326 Front St,8/21/23,2:12:20 PM,PM,101,8,Moving
3,4,42.101760,-75.910613,42.101815,-75.910595,BC Junction 8X,0.000055,34 Prospect Ave (Landmarks: BC JUNCTION),8/21/23,3:07:36 PM,PM,101,8,Idle
4,5,42.156800,-75.896320,42.156811,-75.896378,Front/Northgate,0.000018,1247 Front St,8/21/23,8:34:41 PM,PM,101,8,Moving


In [4]:
def _str_to_date(date_str):
    month, day, year = date_str.split('/')
    year = '20' + year
    return datetime.datetime(int(year), int(month), int(day))
def _str_to_minute(date_str: str, time_str: str):
    month, day, year = date_str.split('/')
    year = '20' + year
    time_str, am_pm = time_str.split(' ')
    hour, minute, _ = time_str.split(':')
    additional_hour = 0
    if am_pm == 'PM' and hour != '12':
        additional_hour = 12
    if am_pm == 'AM' and hour == '12':
        additional_hour = -12
    return datetime.datetime(
        int(year), int(month), int(day),
        int(hour) + additional_hour
    )
def process_data(df, start_date, end_date):
    df['conventional_date'] = df['date'].apply(_str_to_date)
    df['minute'] = df[['date', 'time.x']].apply(lambda arg: _str_to_minute(*arg), axis=1)
    df.sort_values(by='minute', inplace=True)
    df['dummy_size'] = 1
    df['bus_num'] = df['bus_num'].astype(str)
    df = df[(df['conventional_date'] >= start_date) & (df['conventional_date'] <= end_date)]
    return df

processed_data = process_data(
    data,
    datetime.datetime(2023, 8, 15),
    datetime.datetime(2023, 8, 31)
)
processed_data.head()

,id,transaction_latitude,transaction_longitude,bus_stop_latitude,bus_stop_longitude,Stop Name,distance,Address,date,time.x,am_pm,bus_num,Route,movement,conventional_date,minute,dummy_size
33264,33272,42.099271,-76.047858,42.099258,-76.047812,Washington /Main,0.000019,219 Washington Ave,8/15/23,5:54:04 AM,AM,702,35,Moving,2023-08-15,2023-08-15 05:00:00,1
196340,196408,42.095502,-75.922773,42.095570,-75.922737,Leroy/Murray,0.000069,37 Leroy St,8/15/23,6:35:14 AM,AM,746,15,Idle,2023-08-15,2023-08-15 06:00:00,1
181831,181882,42.107093,-75.963378,42.107136,-75.963387,Floral/Cook,0.000043,339 Floral Ave,8/15/23,6:44:04 AM,AM,742,17,Moving,2023-08-15,2023-08-15 06:00:00,1
181830,181881,42.109084,-75.948800,42.109098,-75.948751,Floral/Cleveland,0.000020,44 Floral Ave,8/15/23,6:42:03 AM,AM,742,17,Moving,2023-08-15,2023-08-15 06:00:00,1
196357,196425,42.100480,-75.935573,42.100517,-75.935555,Highland/Beethoven,0.000037,1 Highland Ave,8/15/23,6:39:59 AM,AM,746,15,Idle,2023-08-15,2023-08-15 06:00:00,1


In [5]:
fig = px.scatter_mapbox(
    processed_data, lat="transaction_latitude", lon="transaction_longitude",
    animation_group="bus_num",
    color='bus_num',
    size="dummy_size", size_max=4,
#     animation_frame="conventional_date",
    animation_frame="minute",
    mapbox_style="carto-positron",
#     mapbox_style="open-street-map"
    center={
        'lat': 42.10,
        'lon': -75.95
    }
)
fig.show()

In [6]:
px.scatter_mapbox?

In [7]:
fig = px.density_mapbox(
    processed_data, lat="transaction_latitude", lon="transaction_longitude",
    animation_frame="minute",
#     animation_frame="conventional_date",
    animation_group='bus_num',
    color_continuous_scale='reds',
    range_color=[0,3],
    opacity=0.5,
    radius=7,
    mapbox_style="carto-positron",
#     mapbox_style="open-street-map",
    center={
        'lat': 42.10,
        'lon': -75.95
    }
)
fig.show()

In [8]:
px.density_mapbox?